In [2]:
import requests
from pprint import pprint

In [3]:
prometheus_url = "http://101.202.0.9:30003/api/v1/query"

In [4]:
def prometheus_query(query):
    params = {"query": query}

    response = requests.get(prometheus_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to query Prometheus. Status code: {response.status_code}")
        return None

### CPU

In [5]:
node_address = "192.168.10.5:9100"

query = f'100 - avg(rate(node_cpu_seconds_total{{mode="idle", instance="{node_address}"}}[30s])) * 100'
result = prometheus_query(query)
print(f"CPU Utilization: {result['data']['result'][0]['value'][1]} %")

query = f'node_memory_MemTotal_bytes{{instance="{node_address}"}} - node_memory_MemAvailable_bytes{{instance="{node_address}"}}'
result = prometheus_query(query)
print(f"CPU Memory: {int(result['data']['result'][0]['value'][1])} Byte")

CPU Utilization: 2.885624999937093 %
CPU Memory: 14700978176 Byte


### GPU

In [6]:
hostname = "k8s-node-3"

query = f'DCGM_FI_DEV_GPU_UTIL{{Hostname="{hostname}"}}'
result = prometheus_query(query)
print("GPU Utilization")
for i in result["data"]["result"]:
    print(f"  {i['metric']['UUID']}: {i['value'][1]} %")

query = f'DCGM_FI_DEV_MEM_COPY_UTIL{{Hostname="{hostname}"}}'
result = prometheus_query(query)
print("GPU Memory")
for i in result["data"]["result"]:
    print(f"  {i['metric']['UUID']}: {i['value'][1]} %")

query = f'DCGM_FI_DEV_POWER_USAGE{{Hostname="{hostname}"}}'
result = prometheus_query(query)
print("GPU Power")
for i in result["data"]["result"]:
    print(f"  {i['metric']['UUID']}: {i['value'][1]} W")

query = f'DCGM_FI_DEV_GPU_TEMP{{Hostname="{hostname}"}}'
result = prometheus_query(query)
print("GPU Temperature")
for i in result["data"]["result"]:
    print(f"  {i['metric']['UUID']}: {i['value'][1]} C")

GPU Utilization
  GPU-845e7463-ab58-fbda-345e-2ead5245c75e: 0 %
  GPU-f3a9b66f-587a-ed2b-40ac-73b79dc0505b: 0 %
GPU Memory
  GPU-845e7463-ab58-fbda-345e-2ead5245c75e: 0 %
  GPU-f3a9b66f-587a-ed2b-40ac-73b79dc0505b: 0 %
GPU Power
  GPU-845e7463-ab58-fbda-345e-2ead5245c75e: 9.069 W
  GPU-f3a9b66f-587a-ed2b-40ac-73b79dc0505b: 9.069 W
GPU Temperature
  GPU-845e7463-ab58-fbda-345e-2ead5245c75e: 30 C
  GPU-f3a9b66f-587a-ed2b-40ac-73b79dc0505b: 28 C


### DISK

In [7]:
query = f'rate(node_disk_read_bytes_total{{instance="192.168.10.5:9100",device="sda"}}[30s])'
result = prometheus_query(query)
print(f"Disk Read: {result['data']['result'][0]['value'][1]} Byte")

query = f'rate(node_disk_written_bytes_total{{instance="192.168.10.5:9100",device="sda"}}[30s])'
result = prometheus_query(query)
print(f"Disk Write: {result['data']['result'][0]['value'][1]} Byte")

Disk Read: 0 Byte
Disk Write: 318177.28 Byte
